In [1]:
# Libaries and Tools

import glob
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import astropy.units as u
import datetime

In [ ]:
# Key Varibles

resultsFolder = 'Results/'
saveName = '.Mag.npz'

In [ ]:
# Key Functions
def rollingAvg(dates,values,window):
    
    # Cut window in half
    halfWindow = window/2
    
    avgValues = np.empty(len(values)); avgValues[:] = np.nan
    stdValues = np.empty(len(values)); stdValues[:] = np.nan
    
    # Cycle Through Data
    for i in range(len(dates)):
        
        # Window
        minDate = np.max([dates[i] - halfWindow, dates[0]])
        maxDate = np.min([dates[i] + halfWindow, dates[-1]])
        
        # avg
        avgValues[i] = np.nanmean(values[(dates>minDate)&(dates<maxDate)])
        
        # std
        stdValues[i] = np.nanstd(values[(dates>minDate)&(dates<maxDate)])
        
    return avgValues,stdValues

In [ ]:
# Get Data
dataset = sorted(glob.glob(resultsFolder + '*' + saveName))

In [ ]:
# Stack data

# Open First .npz file and get list of "arrays"
data = np.load(dataset[0], allow_pickle=True)
lst = data.files
    
# Get Data
dates   = data[lst[0]]
pxAreas = data[lst[1]]
MmAreas = data[lst[2]]
sMmean  = data[lst[3]]
usMmean = data[lst[4]]
cMass   = data[lst[5]]

# remaining cases
for i in range(1,len(dataset)):
    
    # Open .npz file and get list of "arrays"
    data = np.load(dataset[i], allow_pickle=True)
    lst = data.files
    
    # Get Data
    newDates   = data[lst[0]]
    newPxAreas = data[lst[1]]
    newMmAreas = data[lst[2]]
    newSMmean  = data[lst[3]]
    newUsMmean = data[lst[4]]
    newCMass   = data[lst[5]]
    
    # Combine Data, Preveting Doubles
    for j in range(len(newDates)):
        if newDates[j] != dates[-1]:
            dates   = np.hstack([dates,   newDates[j:]])
            pxAreas = np.hstack([pxAreas, newPxAreas[j:]])
            MmAreas = np.hstack([MmAreas, newMmAreas[j:]])
            sMmean  = np.hstack([sMmean,  newSMmean[j:]])
            usMmean = np.hstack([usMmean, newUsMmean[j:]])
            cMass   = np.hstack([cMass,   newCMass[j:]])
            break

In [ ]:
print(np.sum(np.isnan(sMmean)))

In [ ]:
maxC = np.empty(len(cMass)); maxC[:] = np.nan
for i in range(len(maxC)):
    maxC[i] = np.max([np.abs(cMass[i].Tx/u.arcsec),np.abs(cMass[i].Ty/u.arcsec)])

In [ ]:
# Sanity Check
print(cMass[0])

In [ ]:
# Projection-corrected Mean Magnetic Field Density
plt.figure(figsize=[5,1.7],dpi=300)
plt.rcParams.update({'font.size': 6})
plt.scatter(dates[maxC<=500],abs(sMmean[maxC<=500]),0.5,'lightsteelblue')
plt.title(r'$|\bar{B}|$ of individual CHs within $\pm500^{\prime\prime}$')
plt.xlabel('Date')
plt.ylabel(r'$|\bar{B}|$ (G)')
plt.grid()
title = 'MagDensityWide.eps'
saveFolder = '../../Figures/DailyCadence/'
if not os.path.exists(saveFolder):
    os.makedirs(saveFolder)
plt.savefig(saveFolder+title)
plt.show()

In [ ]:
# Rolling Avg
windows = [datetime.timedelta(days=60),
           datetime.timedelta(days=120),
           datetime.timedelta(days=240),
           datetime.timedelta(days=365)]

window = datetime.timedelta(days=365)
label=str(window)[:-10]+' sliding window mean'
SmAvg,SmStd = rollingAvg(dates[maxC<=500],abs(sMmean[maxC<=500]),window)

# Projection-corrected Mean Magnetic Field Density - Zoom In
plt.figure(figsize=[5,1.7],dpi=300)
plt.rcParams.update({'font.size': 6})
plt.scatter(dates[maxC<=500],abs(sMmean[maxC<=500]),0.5,'lightsteelblue')
for i in range(len(windows)):
    label=str(windows[i])[:-10] + ' window'
    MmAvg,MmStd = rollingAvg(dates[maxC<=500],abs(sMmean[maxC<=500]),windows[i])
    plt.plot(dates[maxC<=500],MmAvg,lw=.55,label=label)
plt.ylim(-0.5,15.5)
plt.title(r'$|\bar{B}|$ of individual CHs within $\pm500^{\prime\prime}$')
plt.xlabel('Date')
plt.ylabel(r'$|\bar{B}|$ (G)')
plt.grid()
title = 'MagDensityZoom.eps'
saveFolder = '../../Figures/DailyCadence/'
if not os.path.exists(saveFolder):
    os.makedirs(saveFolder)
plt.savefig(saveFolder+title)
plt.show()

In [ ]:
print('Mean absolute Magnetic Field Density in 500\" range:',
      np.nanmean(abs(sMmean[maxC<=500])), '±',
      np.nanstd(abs(sMmean[maxC<=500])), 'G' )

# Remove 2024 Outliers
region1 = (dates<datetime.datetime(2024,5,1)).astype(int) + (dates>datetime.datetime(2024,6,1)).astype(int)
region = (region1 * (maxC<=500).astype(int)).astype(bool)
print('Mean absolute Magnetic Field Density in 500\" range:',
      np.nanmean(abs(sMmean[region])), '±',
      np.nanstd(abs(sMmean[region])), 'G' )

In [ ]:
plt.figure(figsize=[5,1.7],dpi=300)
plt.rcParams.update({'font.size': 6})
plt.scatter(dates,MmAreas,0.5,'lightsteelblue',label='Observed Area')
for i in range(len(windows)):
    label=str(windows[i])[:-10] + ' window'
    MmAvg,MmStd = rollingAvg(dates,MmAreas,windows[i])
    plt.plot(dates,MmAvg,lw=.55,label=label)
#plt.legend()
plt.title('Area in Mm of Individual CHs')
plt.xlabel('Date')
plt.ylabel(r'CH Area (Mm$^2$)')
plt.grid()
title = 'IndividualCHareaMm.eps'
saveFolder = '../../Figures/DailyCadence/'
if not os.path.exists(saveFolder):
    os.makedirs(saveFolder)
plt.savefig(saveFolder+title)
plt.show()

In [ ]:
print('Mean area of all CHs:',
      np.nanmean(MmAreas), '±',
      np.nanstd(MmAreas), 'Mm2' )
print('Median Area of all CHs:',
      np.nanmedian(MmAreas), 'Mm2')
print()
print('Mean area of all CHs:',
      np.nanmean(MmAreas[region1.astype(bool)]), '±',
      np.nanstd(MmAreas[region1.astype(bool)]), 'Mm2' )
print('Median Area of all CHs:',
      np.nanmedian(MmAreas[region1.astype(bool)]), 'Mm2')

In [ ]:
# Number of CHs
dts,Counts = np.unique(dates,return_counts=True)

plt.figure(figsize=[5,1.7],dpi=300)
plt.rcParams.update({'font.size': 6})
plt.bar(dts,Counts)#,1)
plt.title('Number of individual CHs')
plt.xlabel('Observation, organized by Date')
plt.ylabel('Number of CHs')
plt.ylim([0,21])
plt.grid()
title = 'NumberOfCHsZoom.eps'
saveFolder = '../../Figures/DailyCadence/'
if not os.path.exists(saveFolder):
    os.makedirs(saveFolder)
plt.savefig(saveFolder+title)
plt.show()

# Get dates of outliers
for i in range(len(Counts)):
    
    if Counts[i]>=20:
        print(dts[i],Counts[i])

In [ ]:
# Number of CHs
dts,Counts = np.unique(dates,return_counts=True)

plt.figure(figsize=[5,1.7],dpi=300)
plt.rcParams.update({'font.size': 6})
plt.bar(dts,Counts)#,1)
plt.title('Number of individual CHs')
plt.xlabel('Observation, organized by Date')
plt.ylabel('Number of CHs')
plt.ylim(-0.5,20.5)
plt.grid()
title = 'NumberOfCHsZoom.eps'
saveFolder = '../../Figures/DailyCadence/'
if not os.path.exists(saveFolder):
    os.makedirs(saveFolder)
plt.savefig(saveFolder+title)
plt.show()

In [ ]:
plt.figure(figsize=[5,1.7],dpi=300)
plt.rcParams.update({'font.size': 6})
plt.scatter(dates[maxC<=500],MmAreas[maxC<=500],0.5,'lightsteelblue')
for i in range(len(windows)):
    label=str(windows[i])[:-10] + ' window'
    MmAvg,MmStd = rollingAvg(dates[maxC<=500],MmAreas[maxC<=500],windows[i])
    plt.plot(dates[maxC<=500],MmAvg,lw=.55,label=label)
plt.title(r'Area of Individual CHs within $\pm500^{\prime\prime}$')
plt.xlabel('Date')
plt.ylabel(r'CH Area (Mm$^2$)')
plt.grid()
title = 'IndividualSquareCHareaMm.eps'
saveFolder = '../../Figures/DailyCadence/'
if not os.path.exists(saveFolder):
    os.makedirs(saveFolder)
plt.savefig(saveFolder+title)
plt.show()

In [ ]:
# Rolling Avg
MmAvg,MmStd = rollingAvg(dates[maxC<=500],MmAreas[maxC<=500],window)


plt.figure(figsize=[5,1.7],dpi=300)
plt.rcParams.update({'font.size': 6})
plt.scatter(dates[maxC<=500],MmAreas[maxC<=500],0.5,'lightsteelblue')
for i in range(len(windows)):
    label=str(windows[i])[:-10] + ' window'
    MmAvg,MmStd = rollingAvg(dates[maxC<=500],MmAreas[maxC<=500],windows[i])
    plt.plot(dates[maxC<=500],MmAvg,lw=.55,label=label)
plt.title(r'Area of Individual CHs within $\pm500^{\prime\prime}$')
plt.xlabel('Date')
plt.ylabel(r'CH Area (Mm$^2$)')
plt.ylim(-1000,150000)
plt.grid()
title = 'IndividualSquareCHareaMmZoom.eps'
saveFolder = '../../Figures/DailyCadence/'
if not os.path.exists(saveFolder):
    os.makedirs(saveFolder)
plt.savefig(saveFolder+title)
plt.show()